In [ ]:
%%capture
##Update python version to python 3.7. This is required as latest python does not have tensorflow gpu 1.15 support. Tensorflow 1.xx is needed for this training
!sudo apt-get install python3.7
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 3
!sudo apt-get install python3.7-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!sudo python3.7 get-pip.py
!sudo apt-get install python3.7-distutils
!pip install --upgrade "protobuf<=3.20.1"
!pip install Pillow

In [ ]:
%%capture
#Get tensorflow model library
!git clone https://github.com/tensorflow/models.git
! pip install contextlib2
import os
new_python_path = (os.environ.get("PYTHONPATH") or '') + ":models/research/slim"
%env PYTHONPATH=$new_python_path
!pip install tf-slim

In [ ]:
#TF-Slim requires tensorflow 1.x to run. However, google colab no longer supports tensorflow 1.x. Workaround, we need to manually
#install tensorflow 1.xx using pip. Ignore the error regarding gast version incompatibility
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.15
!apt install --allow-change-held-packages libcudnn7=7.4.1.5-1+cuda10.0

In [ ]:
#Get dataset from visual wake words
! python models/research/slim/download_and_convert_data.py \
--logtostderr \
--dataset_name=visualwakewords \
--dataset_dir=person_detection_dataset \
--foreground_class_of_interest='person' \
--small_object_area_threshold=0.005

In [ ]:
#Uncomment the three lines below to download pre-trained weight. The weight is trained up to 100K steps, thus it will continue from 100K points, up to 1 Million steps

# !wget -O person_detection_train.zip https://www.dropbox.com/s/73u1ddqp8hmx8pt/person_detection_train.zip?dl=0
# !unzip person_detection_train.zip
# !rm -rf person_detection_train.zip

#Train the model. Person detection model is trained for 1Million steps. 
! python models/research/slim/train_image_classifier.py \
    --alsologtostderr \
    --dataset_name=visualwakewords \
    --dataset_dir=person_detection_dataset \
    --dataset_split_name=train \
    --train_image_size=96 \
    --use_grayscale=True \
    --preprocessing_name=mobilenet_v1 \
    --model_name=mobilenet_v1_025 \
    --train_dir=person_detection_train \
    --save_summaries_secs=300 \
    --learning_rate=0.045 \
    --clone_on_cpu=True \
    --label_smoothing=0.1 \
    --learning_rate_decay_factor=0.98 \
    --num_epochs_per_decay=2.5 \
    --moving_average_decay=0.9999 \
    --batch_size=96 \
    --max_number_of_steps=1000000

In [ ]:
#Evaluate your model based on your checkpoints. Ensure that the checkpoint is based on your latest steps (e.g; model.ckpt-100000 for 100K steps)
! python models/research/slim/eval_image_classifier.py \
    --alsologtostderr \
    --dataset_name=visualwakewords \
    --dataset_dir=person_detection_dataset \
    --dataset_split_name=val \
    --eval_image_size=96 \
    --use_grayscale=True \
    --preprocessing_name=mobilenet_v1 \
    --model_name=mobilenet_v1_025 \
    --train_dir=person_detection_train \
    --checkpoint_path=person_detection_train/model.ckpt-100000

In [ ]:
#Generate model graph 
! python models/research/slim/export_inference_graph.py \
    --alsologtostderr \
    --dataset_name=visualwakewords \
    --image_size=96 \
    --use_grayscale=True \
    --model_name=mobilenet_v1_025 \
    --output_file=person_detection_graph.pb

In [ ]:
#Generate frozen graph based on checkpoint. Ensure that the checkpoint is based on your latest steps (e.g; model.ckpt-100000 for 100K steps)
! git clone -b r1.15 https://github.com/tensorflow/tensorflow.git
! python tensorflow/tensorflow/python/tools/freeze_graph.py \
--input_graph=person_detection_graph.pb \
--input_checkpoint=person_detection_train/model.ckpt-100000 \
--input_binary=true \
--output_node_names=MobilenetV1/Predictions/Reshape_1 \
--output_graph=person_detection_frozen_graph.pb

In [ ]:
#Generate quantized model tflite
import tensorflow.compat.v1 as tf
import io
import PIL
import numpy as np

def representative_dataset_gen():

  record_iterator = tf.python_io.tf_record_iterator(path='person_detection_dataset/val.record-00000-of-00010')

  for _ in range(250):
    string_record = next(record_iterator)
    example = tf.train.Example()
    example.ParseFromString(string_record)
    image_stream = io.BytesIO(example.features.feature['image/encoded'].bytes_list.value[0])
    image = PIL.Image.open(image_stream)
    image = image.resize((96, 96))
    image = image.convert('L')
    array = np.array(image)
    array = np.expand_dims(array, axis=2)
    array = np.expand_dims(array, axis=0)
    array = ((array / 127.5) - 1.0).astype(np.float32)
    yield([array])

converter = tf.lite.TFLiteConverter.from_frozen_graph('person_detection_frozen_graph.pb',
['input'], ['MobilenetV1/Predictions/Reshape_1'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()
open("person_detection_model.tflite", "wb").write(tflite_quant_model)

In [ ]:
#Rename tflite model following <architecture>_<application>.tflite. This will be used in TinyML Generator for accelerator and model generation
!mv person_detection_model.tflite mobilenetv1_person_detect.tflite 